In [1]:
# !pip install ta

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import datetime as dt
import math

import ta

from tqdm.contrib.telegram import tqdm, trange
from scipy import optimize
# import winsound

# # for using GPU
# from numba import jit, cuda

# to measure exec time
# from timeit import default_timer as timer

import json

In [3]:
# save as json
# import json
# import datetime as dt

def save_as_json(inp_dict, output_folder, indent = 4, output_filename_prefix = 'output'):
    now = dt.datetime.now()
    ret = [now.month, now.day, now.hour, now.minute]
    ret = ['0' + str(i) if len(str(i)) == 1 else i for i in ret]
    now_str = "{}{}{}-{}{}-{}".format(now.year, ret[0], ret[1], ret[2], ret[3], now.second)
    output_filename = f'{output_filename_prefix}_{now_str}.json'
    output_path = f'{output_folder}\\{output_filename}'
    
    out_file = open(output_path, "w")
    json.dump(inp_dict, out_file, indent = indent)
    out_file.close()

In [4]:
# import datetime as dt
# import pandas as pd

def save_as_csv(input_df, output_folder, output_filename_prefix = 'output'):
#     output_folder = r'C:\Users\priva\OneDrive - HKUST Connect\Stock\Data\temp outputs'
    now = dt.datetime.now()
    ret = [now.month, now.day, now.hour, now.minute]
    ret = ['0' + str(i) if len(str(i)) == 1 else i for i in ret]
    now_str = "{}{}{}-{}{}-{}".format(now.year, ret[0], ret[1], ret[2], ret[3], now.second)
    output_filename = f'{output_filename_prefix}_{now_str}.csv'
    
    output_link = f'{output_folder}\\{output_filename}'
    input_df.to_csv(output_link)
    print(f"Output file is saved to \n{output_folder}\nFilename: {output_filename}")
    return output_link

In [5]:
def send_to_telegram(message, token, chat_id):
    import requests
    apiURL = f'https://api.telegram.org/bot{token}/sendMessage'

    try:
        response = requests.post(apiURL, json={'chat_id': chat_id, 'text': message})
        print(response.text)
    except Exception as e:
        print(e)

# Get Stock Data

In [9]:
df = pd.read_csv(r'')
df = df.drop(columns = ['Unnamed: 0'])
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,1928-01-03,17.760000,17.760000,17.760000,17.760000,17.760000,0
1,1928-01-04,17.719999,17.719999,17.719999,17.719999,17.719999,0
2,1928-01-05,17.549999,17.549999,17.549999,17.549999,17.549999,0
3,1928-01-06,17.660000,17.660000,17.660000,17.660000,17.660000,0
4,1928-01-09,17.500000,17.500000,17.500000,17.500000,17.500000,0
...,...,...,...,...,...,...,...
23815,2022-10-21,3657.100098,3757.889893,3647.419922,3752.750000,3752.750000,5078020000
23816,2022-10-24,3762.010010,3810.739990,3741.649902,3797.340088,3797.340088,4747930000
23817,2022-10-25,3799.439941,3862.850098,3799.439941,3859.110107,3859.110107,4843120000
23818,2022-10-26,3825.969971,3886.149902,3824.070068,3830.600098,3830.600098,4817310000


# Data Preprocessing

In [10]:
df = df.dropna()

In [11]:
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,1928-01-03,17.760000,17.760000,17.760000,17.760000,17.760000,0
1,1928-01-04,17.719999,17.719999,17.719999,17.719999,17.719999,0
2,1928-01-05,17.549999,17.549999,17.549999,17.549999,17.549999,0
3,1928-01-06,17.660000,17.660000,17.660000,17.660000,17.660000,0
4,1928-01-09,17.500000,17.500000,17.500000,17.500000,17.500000,0
...,...,...,...,...,...,...,...
23815,2022-10-21,3657.100098,3757.889893,3647.419922,3752.750000,3752.750000,5078020000
23816,2022-10-24,3762.010010,3810.739990,3741.649902,3797.340088,3797.340088,4747930000
23817,2022-10-25,3799.439941,3862.850098,3799.439941,3859.110107,3859.110107,4843120000
23818,2022-10-26,3825.969971,3886.149902,3824.070068,3830.600098,3830.600098,4817310000


In [12]:
# replace Adj Close with Close
# drop the Adj Close column
df['Close'] = df['Adj Close']
df = df.drop(columns = ['Adj Close'])
df

,Date,Open,High,Low,Close,Volume
0,1928-01-03,17.760000,17.760000,17.760000,17.760000,0
1,1928-01-04,17.719999,17.719999,17.719999,17.719999,0
2,1928-01-05,17.549999,17.549999,17.549999,17.549999,0
3,1928-01-06,17.660000,17.660000,17.660000,17.660000,0
4,1928-01-09,17.500000,17.500000,17.500000,17.500000,0
...,...,...,...,...,...,...
23815,2022-10-21,3657.100098,3757.889893,3647.419922,3752.750000,5078020000
23816,2022-10-24,3762.010010,3810.739990,3741.649902,3797.340088,4747930000
23817,2022-10-25,3799.439941,3862.850098,3799.439941,3859.110107,4843120000
23818,2022-10-26,3825.969971,3886.149902,3824.070068,3830.600098,4817310000


In [13]:
df['Close_ln'] = np.log(df['Close'])
df

,Date,Open,High,Low,Close,Volume,Close_ln
0,1928-01-03,17.760000,17.760000,17.760000,17.760000,0,2.876949
1,1928-01-04,17.719999,17.719999,17.719999,17.719999,0,2.874694
2,1928-01-05,17.549999,17.549999,17.549999,17.549999,0,2.865054
3,1928-01-06,17.660000,17.660000,17.660000,17.660000,0,2.871302
4,1928-01-09,17.500000,17.500000,17.500000,17.500000,0,2.862201
...,...,...,...,...,...,...,...
23815,2022-10-21,3657.100098,3757.889893,3647.419922,3752.750000,5078020000,8.230244
23816,2022-10-24,3762.010010,3810.739990,3741.649902,3797.340088,4747930000,8.242056
23817,2022-10-25,3799.439941,3862.850098,3799.439941,3859.110107,4843120000,8.258192
23818,2022-10-26,3825.969971,3886.149902,3824.070068,3830.600098,4817310000,8.250777


# Backtesting for Training and Testing

In [14]:
# Version 3

# @jit(target_backend='cuda') 
def backtest(df, long_rules, short_rules, long_only = True, \
             initial_capital=100000, commission_percent=0.5, risk_free_rate = 5, \
             buy_next_day=True, date='Date', buy_at = 'Close',\
             signals='Signals', shares='Shares', cash='Cash', stock='Stock', equity='Equity', 
             action_completed = "Action Completed", trading_days = 250):
    
    global mean_delta_equity_annual, sd_delta_equity_annual
    
    df = df.copy()
    df.reset_index(drop=False, inplace=True)
    
    # Assume no short selling allowed
    # ie sell holing stocks whenever a short signal is triggered
    
    # Check for any conflicts between long_rules and short_rules
    for i in range(len(long_rules)):
        # Cannot long/short at the same time
        if abs(long_rules[i])>0 and abs(short_rules[i])>0:
            return 'Signal problem(s) found at index {}'.format(i)
    
    # Init
    action_delay = 1 if buy_next_day else 0
    
    df[signals] = [long_rules[i]+short_rules[i] for i in range(len(long_rules))]  
    df.loc[0,cash] = initial_capital
    df.loc[0,stock] = 0
    df.loc[0,shares] = 0
    df.loc[0,action_completed] = 0
    
    def buy_shares(cash, price, commission_percent):
        shares = cash//price
        if price == 0:
            return None
        while shares*price*(1+commission_percent/100)>cash and shares > 0:
            shares -= 1
        return shares
    
    txn_list = []
    # Iterate through the df
    for r in range(1, len(df)):

        # here 2022-11-12 
        # debug for conflicts between the proposed action and new signal; yet selling (exit long) overrides
        proposed_action = df.loc[r-action_delay, signals]
        new_signal = df.loc[r, signals]
        
        # Long triggered
        if proposed_action > 0 and not(new_signal < 0) :
#         if df.loc[r-action_delay, signals] > 0 :
            if df.loc[r-1, shares] == 0:
                df.loc[r,shares] = buy_shares(cash = df.loc[r-1,cash],\
                                             price = df.loc[r,buy_at],\
                                             commission_percent = commission_percent)
                if df.loc[r,shares] == None:
                    print('Error: at ' + str(r))
                
                df.loc[r,stock] = df.loc[r,shares]*df.loc[r,buy_at]
                df.loc[r,cash] = df.loc[r-1,cash] - df.loc[r,stock]
                
                # update txn_list only if a purchase is really completed
                if df.loc[r,shares] > 0:
                    txn_list.append([r, 1] + list(df.loc[r, [date, shares, stock, cash]]))
                    df.loc[r,action_completed] = 1
                else:
                    df.loc[r,action_completed] = 0
            else:
                # Hold/No Action
                df.loc[r,shares] = df.loc[r-1, shares]
                df.loc[r,cash] = df.loc[r-1,cash]
                df.loc[r,stock] = df.loc[r-1,stock]
                df.loc[r,action_completed] = 0
            
        # Short triggered/ Selling triggered for Long-Only Appraoch
        # Selling (exit long) overrides
        elif proposed_action < 0:
#         elif df.loc[r-action_delay, signals] < 0:
            df.loc[r,shares] = 0
            df.loc[r,stock] = 0
            df.loc[r,cash] = df.loc[r-1,cash] + df.loc[r-1,shares]*df.loc[r,buy_at]*(1-commission_percent/100)
            # update txn_list only if a sale is really completed (ie previously holding the stock)
            if df.loc[r-1,shares] > 0:
                txn_list.append([r, -1] + list(df.loc[r, [date, shares, stock, cash]]))
            
            df.loc[r,action_completed] = 0 if df.loc[r,shares] == df.loc[r-1,shares] else -1

        # Hold/No Action
        else:
            df.loc[r,shares] = df.loc[r-1,shares]
            df.loc[r,stock] = df.loc[r,shares]*df.loc[r,buy_at]
            df.loc[r,cash] = df.loc[r-1,cash]
            df.loc[r,action_completed] = 0
            
    df[equity] = df[cash] + df[stock] 
       
    # -----------------------------------------------------------------------------------------------------
    def get_pct_change(x_prev, x_new, scale = 100):
        if x_prev == 0.:
            ret_sgn = np.sign([x_prev, x_new]).prod()
            return -math.inf if ret_sgn < 0 else math.inf
        else: 
            return scale*(x_new/x_prev-1)
    
    # consider each PAIR of buying and selling

    txn_df = pd.DataFrame(txn_list, columns = ['idx', 'action', 'date', 'shares', 'stock', 'cash'])
    
    # buy-and-hold return % 
    # corrected on 2022-11-10 1639
    bnh_pct = None
    
    # Sharpe
    dayscount_backtest = len(df)    # neglect days without prices, eg public holidays
    df[f'delta_{equity}'] = df[equity] / df[equity].shift(1) - 1
    
    mean_delta_equity = df[f'delta_{equity}'].mean()
    mean_delta_equity_annual = mean_delta_equity * trading_days
    sd_delta_equity_annual = df[f'delta_{equity}'].std(ddof=0) * np.sqrt(trading_days)
    sd_delta_equity_annual = 10**-300 if sd_delta_equity_annual == 0. else sd_delta_equity_annual    # penalty; to escape DivisionByZero error 
    sharpe_delta_equity_annual = (mean_delta_equity_annual - risk_free_rate) / sd_delta_equity_annual
    
    # Sortino
    sd_downside_delta_equity_annual = df[f'delta_{equity}'][df[f'delta_{equity}'] <= mean_delta_equity].std(ddof=0) * np.sqrt(trading_days)
    sd_downside_delta_equity_annual = 10**-300 if sd_downside_delta_equity_annual == 0. else sd_downside_delta_equity_annual    # penalty; to escape DivisionByZero error
    sortino_delta_equity_annual = (mean_delta_equity_annual - risk_free_rate) / sd_downside_delta_equity_annual
    
    # here 2022-12-07
    # net profit %
    profit_net_pct = get_pct_change(x_prev = df.loc[df.index[0],equity], 
                                    x_new = df.loc[df.index[-1],equity], 
                                    scale = 100)
    
    if len(txn_df) >= 1:
        bnh_cash = txn_df.loc[0, 'cash']
        bnh_shares = txn_df.loc[0, 'shares']
        bnh_stock = txn_df.loc[0, 'stock']
        bnh_last_price = df.loc[len(df)-1, buy_at]
        
        bnh_pct = get_pct_change(x_prev = bnh_cash + bnh_stock, 
                                 x_new = bnh_cash + bnh_shares * bnh_last_price, 
                                 scale = 100)
        
    if len(txn_df) >=2:
        txn_df['equity'] = txn_df['cash'] + txn_df['stock']

        # return of each pair of transaction
        i = 0
        txn_df['return_txnpair'] = np.nan
        txn_df['return_pct_txnpair'] = np.nan
        while i <= len(txn_list)-2:  
            txn_df.loc[i, 'return_txnpair'] = txn_df.loc[i+1, 'equity'] - txn_df.loc[i, 'equity']
            txn_df.loc[i, 'return_pct_txnpair'] = get_pct_change(x_prev = txn_df.loc[i, 'equity'], 
                                                                 x_new = txn_df.loc[i+1, 'equity'], 
                                                                 scale = 100)
            i+=2
        
#         # net profit %
#         profit_net_pct = get_pct_change(x_prev = df.loc[df.index[0],equity], 
#                                         x_new = df.loc[df.index[-1],equity], 
#                                         scale = 100)
        
#         print(f'profit_net_pct: {profit_net_pct}')
        
        # gross profit, gross loss, and their %
        profit_gross = txn_df[txn_df['return_txnpair'] > 0]['return_txnpair'].sum()
        profit_gross_pct = get_pct_change(x_prev = initial_capital, 
                                          x_new = initial_capital+profit_gross, 
                                          scale = 100)
#         print(f'profit_net_pct: {profit_net_pct}')
        
        loss_gross = txn_df[txn_df['return_txnpair'] <= 0]['return_txnpair'].sum()
        loss_gross_pct = get_pct_change(x_prev = initial_capital, 
                                        x_new = initial_capital+loss_gross, 
                                        scale = 100)
#         print(f'loss_gross_pct: {loss_gross_pct}')
        
        # profit factor
        profit_factor = profit_gross / loss_gross if abs(loss_gross) > 0 else 10**9    # prevent DivisionByZero
        
#         print(f'profit_factor: {profit_factor}')

        # max run-up and max draw-down
        runup_pct_max = txn_df['return_pct_txnpair'].max()
        drawdown_pct_max = txn_df['return_pct_txnpair'].min()    # using min because the drawdown % is neg
        
#         print(f'runup_pct_max: {runup_pct_max}')
#         print(f'drawdown_pct_max: {drawdown_pct_max}')

        # Sharpe & Sortino; Avg Trade (geo mean) (mean_return_pct_annual)
        # Also the avg winning trade and avg losing trade
        dayscount_backtest = len(df)    # neglect days without prices, eg public holidays

        mean_return_pct_annual = txn_df['return_pct_txnpair'].dropna().apply(lambda x: x/100 + 1)
        mean_return_pct_annual = np.power(mean_return_pct_annual.product(), 250/dayscount_backtest)-1    # assume 250 trading days annually
        
#         # SUPERSEDED
#         # Wrong formula
#         sd_return_pct_annual = txn_df['return_pct_txnpair'].std(ddof = 0)
        
#         sd_return_pct_annual *= np.sqrt(dayscount_backtest)    # assume daily data input, and so annualized by multiplying sqrt(days coount)
        
#         sd_downside_return_pct_annual = txn_df[txn_df['return_pct_txnpair'] <= 0]['return_pct_txnpair'].std(ddof = 0)
#         sd_downside_return_pct_annual *= np.sqrt(dayscount_backtest)    # assume daily data input, and so annualized by multiplying sqrt(days coount)
        
#         sharpe_annual = None if math.isnan(sd_return_pct_annual) or sd_return_pct_annual == 0. else \
#         (mean_return_pct_annual - risk_free_rate)/sd_return_pct_annual
        
#         sortino_annual = None if math.isnan(sd_downside_return_pct_annual) or sd_downside_return_pct_annual == 0. else \
#         (mean_return_pct_annual - risk_free_rate)/sd_downside_return_pct_annual

        # Avg winning trade
        # here debug 2022-11-04 WIP
        avg_loss_pct_annual = None
        avg_win_pct_annual = None
        avg_wintoloss_ratio_annual = None
        
#         if math.isnan(mean_return_pct_annual[mean_return_pct_annual > 0].prod()):
#             avg_win_pct_annual = None
#         else:
#             avg_win_pct_annual = np.power(mean_return_pct_annual[mean_return_pct_annual > 0].prod(), 250/dayscount_backtest)-1
        
#         # Avg losing trade
#         if math.isnan(mean_return_pct_annual[mean_return_pct_annual <= 0]):
#             avg_loss_pct_annual = None
#         else:
#             avg_loss_pct_annual = np.power(mean_return_pct_annual[mean_return_pct_annual <= 0].prod(), 250/dayscount_backtest)-1
            
#         # Avg Win-to-Loss Ratio
#         if avg_loss_pct_annual == None or avg_win_pct_annual == None:
#             avg_wintoloss_ratio_annual = None
#         elif avg_loss_pct_annual == 0:
#             avg_wintoloss_ratio_annual = math.inf
#         else:
#             avg_wintoloss_ratio_annual = avg_win_pct_annual/avg_loss_pct_annual

        # Max Contracts Held (max_shares)
        max_shares = txn_df['shares'].max()

        # Number Winning Trades and Losing Trades and win rate (win_pct)
        txn_pairs = txn_df['return_pct_txnpair'].dropna()
        win_count = len(txn_pairs[txn_pairs>0])
        loss_count = len(txn_pairs[txn_pairs<=0])
        win_pct = 100*win_count/len(txn_pairs)  
        
#         print(f'win_count, loss_count, win_pct: {win_count}, {loss_count}, {win_pct}')

        txn_summary = {'bnh_pct': bnh_pct, 
                       'profit_net_pct': profit_net_pct, 
                       'profit_gross_pct': profit_gross_pct, 
                       'loss_gross_pct': loss_gross_pct, 
                       'profit_factor': profit_factor, 
                       'runup_pct_max': runup_pct_max, 
                       'drawdown_pct_max': drawdown_pct_max, 
                       'mean_return_pct_annual': mean_return_pct_annual, 
                       'sd_delta_equity_annual': sd_delta_equity_annual, 
                       'sd_downside_delta_equity_annual': sd_downside_delta_equity_annual, 
                       'risk_free_rate': risk_free_rate,
                       'sharpe_delta_equity_annual': sharpe_delta_equity_annual, 
                       'sortino_delta_equity_annual': sortino_delta_equity_annual, 
                       'avg_win_pct_annual': avg_win_pct_annual, 
                       'avg_loss_pct_annual': avg_loss_pct_annual, 
                       'avg_wintoloss_ratio_annual': avg_wintoloss_ratio_annual, 
                       'max_shares': max_shares, 
                       'win_count': win_count, 
                       'loss_count':loss_count,
                       'win_pct': win_pct, 
                       'txn_list': txn_list}
    else:
        win_count = 0
        loss_count = 0
        txn_summary = {'bnh_pct': bnh_pct, 
                       'profit_net_pct': profit_net_pct, 
                       'profit_gross_pct': None, 
                       'loss_gross_pct': None, 
                       'profit_factor': None, 
                       'runup_pct_max': None, 
                       'drawdown_pct_max': None, 
                       'mean_return_pct_annual': None, 
                       'sd_delta_equity_annual': sd_delta_equity_annual, 
                       'sd_downside_delta_equity_annual': sd_downside_delta_equity_annual, 
                       'risk_free_rate': risk_free_rate,
                       'sharpe_delta_equity_annual': sharpe_delta_equity_annual, 
                       'sortino_delta_equity_annual': sortino_delta_equity_annual, 
                       'avg_win_pct_annual': 0, 
                       'avg_loss_pct_annual': None, 
                       'avg_wintoloss_ratio_annual': None, 
                       'max_shares': 0, 
                       'win_count': win_count, 
                       'loss_count':loss_count,
                       'win_pct': 0, 
                       'txn_list': txn_list}
    

    
#     # testing, wait for debug
#     # debug here 2022-10-31 0204
    
#     txn_df = pd.DataFrame()
#     txn_summary = dict()
    
    return (df, txn_df, txn_summary)
    

In [22]:
# @jit(target_backend='cuda')
def add_dc(df, dc_source, dc_len, bound_pct = 0, bound = 'both'):
    
    df = df.copy()
    
    if bound.lower() == 'upper':
        df['DC-up({}, {})'.format(dc_source, dc_len)] = df[dc_source].rolling(dc_len).max().apply(lambda x: (1+bound_pct)*x)
    
    elif bound.lower() == 'lower':
        df['DC-low({}, {})'.format(dc_source, dc_len)] = df[dc_source].rolling(dc_len).min().apply(lambda x: (1-bound_pct)*x)
    
    elif bound == 'both':
        df['DC-up({}, {})'.format(dc_source, dc_len)] = df[dc_source].rolling(dc_len).max().apply(lambda x: (1+bound_pct)*x)
        df['DC-low({}, {})'.format(dc_source, dc_len)] = df[dc_source].rolling(dc_len).min().apply(lambda x: (1-bound_pct)*x)
    else:
        assert False, '\'bound\' is expected to be either \'upper\', \'lower\', or \'both\'.'
    return df

In [23]:
def add_kama(df, src_colname, delta_len = 10, fast_len = 2, slow_len = 30, with_kama_mean = True, kama_mean_len = 25, fillna = False):
    df = df.copy()
    kama = ta.momentum.kama(close=df[src_colname], 
                            window = delta_len, 
                            pow1 = fast_len, 
                            pow2 = slow_len, 
                            fillna = False)
    df[f'KAMA({src_colname}, {delta_len}, {fast_len}, {slow_len})'] = kama
    if with_kama_mean:
        # shifted KAMA by the mean of residual within a rolling window
        rsd = df[src_colname] - kama                          # kama here is the original KAMA
        rsd_mean = rsd.rolling(kama_mean_len).mean()
        kama_mean = kama + rsd_mean                           # shifted KAMA-Mean
        df[f'KAMA-Mean({src_colname}, {delta_len}, {fast_len}, {slow_len}, {kama_mean_len})'] = kama_mean  
    return df

In [24]:
def delta(inp_list, normalize):
    # last value - start value
    ret = inp_list[-1] - inp_list[0]
    if normalize:
        ret/=len(inp_list)
    return ret

In [25]:
# Caution: 
# 1. df_temp is to be assigned OUTSIDE the function below
# 2. input of the function below CANNOT be changed, or otherwise optimization cannot be performed.

# @jit(target_backend='cuda')
def objective_func(x):
    global df_temp, df_bt, txn_df, txn_summary, risk_free_rate
    
    # ==============================================================================================
    # KAMA Only
    # here 2022-11-29 1211
    n_delta, n_fast, n_slow, delta_lim_up, delta_lim_dn, delta_lim_len = x
    
    src_name = 'Close_ln'
    n_delta = int(round(n_delta,0))    # ensure integer
    n_fast = int(round(n_fast,0))      # ensure integer
    n_slow = int(round(n_slow,0))      # ensure integer
    delta_lim_len = int(round(delta_lim_len, 0))    # ensure integer
    
    assert df_temp.index[0] - extra_len_init >= df.index[0], \
    'Insufficient prior time series data for extra_len_init. {} more is needed.'.format(df.index[0] - rows_train[0] + extra_len_init)
    
    df_temp = df.loc[df_temp.index[0]-extra_len_init : df_temp.index[-1], ]    
    # ....................................
    
    df_temp = add_kama(df = df_temp, 
                       src_colname = src_name, 
                       delta_len = n_delta, 
                       fast_len = n_fast, 
                       slow_len = n_slow, 
                       with_kama_mean = False, 
                       kama_mean_len = None, 
                       fillna = False)
    
    ub_col = f'KAMA({src_name}, {n_delta}, {n_fast}, {n_slow})'
    lb_col = ub_col
    kama_delta_col = f'delta({ub_col}, {delta_lim_len})'
    
    kama_delta = df_temp[ub_col].rolling(delta_lim_len).apply(lambda x: delta(list(x), normalize = True))    # Caution: delta of KAMA, not of KAMA-Mean
    df_temp[kama_delta_col] = kama_delta    # Caution: delta of KAMA, not of KAMA-Mean

    # drop the first extra_len_init rows of data
    # these rows are just needed for prevention of NaN values in the indicator column(s) (ie KAMA and KAMA-Mean)
    df_temp.drop(df_temp.index[:extra_len_init], inplace=True)
    
    # buying / selling actions 
    long_actions = df_temp[kama_delta_col] - delta_lim_up
    long_actions = long_actions.apply(lambda x: 1 if x > 0 else None)
    df_temp['long_actions'] = long_actions  

    short_actions = df_temp[kama_delta_col] - delta_lim_dn
    short_actions = short_actions.apply(lambda x: -1 if x < 0 else None)
    df_temp['short_actions'] = short_actions    
    
#     # ==============================================================================================
#     # KAMA-enter DC-exit
#     # here 2022-11-24 1239
#     n_delta, n_fast, n_slow, delta_lim_up, delta_lim_len, n_dn = x
    
#     src_name = 'Close_ln'
#     n_delta = int(round(n_delta,0))    # ensure integer
#     n_fast = int(round(n_fast,0))      # ensure integer
#     n_slow = int(round(n_slow,0))      # ensure integer
#     delta_lim_len = int(round(delta_lim_len, 0))    # ensure integer
#     n_dn = int(round(n_dn,0))    # ensure integer
    
#     assert df_temp.index[0] - extra_len_init >= df.index[0], \
#     'Insufficient prior time series data for extra_len_init. {} more is needed.'.format(df.index[0] - rows_train[0] + extra_len_init)
    
#     df_temp = df.loc[df_temp.index[0]-extra_len_init : df_temp.index[-1], ]    
#     # ....................................
    
#     df_temp = add_kama(df = df_temp, 
#                        src_colname = src_name, 
#                        delta_len = n_delta, 
#                        fast_len = n_fast, 
#                        slow_len = n_slow, 
#                        with_kama_mean = False, 
#                        kama_mean_len = None, 
#                        fillna = False)
    
#     df_temp = add_dc(df = df_temp, 
#                      dc_source = src_name, 
#                      dc_len = n_dn, 
#                      bound_pct = 0, 
#                      bound = 'lower')
    
#     ub_col = f'KAMA({src_name}, {n_delta}, {n_fast}, {n_slow})'
#     lb_col = f'DC-low({src_name}, {n_dn})'
#     kama_delta_col = f'delta({ub_col}, {delta_lim_len})'
    
#     kama_delta = df_temp[ub_col].rolling(delta_lim_len).apply(lambda x: delta(list(x), normalize = True))    # Caution: delta of KAMA, not of KAMA-Mean
#     df_temp[kama_delta_col] = kama_delta    # Caution: delta of KAMA, not of KAMA-Mean

#     # drop the first extra_len_init rows of data
#     # these rows are just needed for prevention of NaN values in the indicator column(s) (ie KAMA and KAMA-Mean)
#     df_temp.drop(df_temp.index[:extra_len_init], inplace=True)
    
#     # buying / selling actions 
#     long_actions = df_temp[kama_delta_col] - delta_lim_up
#     long_actions = long_actions.apply(lambda x: 1 if x > 0 else None)
#     df_temp['long_actions'] = long_actions  

#     short_actions = (df_temp[src_name] / df_temp[lb_col].shift(1) - 1) - 0
#     short_actions = short_actions.apply(lambda x: -1 if x < 0 else None)
#     df_temp['short_actions'] = short_actions

#     # ==============================================================================================
#     # KAMA-Mean
#     # here 2022-11-05 0951
#     n_delta, n_fast, n_slow, delta_lim_up, delta_lim_dn, delta_lim_len, kama_mean_len = x
    
#     src_name = 'Close_ln'
#     n_delta = int(round(n_delta,0))    # ensure integer
#     n_fast = int(round(n_fast,0))      # ensure integer
#     n_slow = int(round(n_slow,0))      # ensure integer
#     kama_mean_len = int(round(kama_mean_len, 0))    # ensure integer
#     delta_lim_len = int(round(delta_lim_len, 0))    # ensure integer
    
#     # ....................................
#     # here 2022-11-13 1351
#     # consider extra prev data to generate values of KAMA-Mean in the beginning days so as to prevent NaN in these days
    
#     assert df_temp.index[0] - extra_len_init >= df.index[0], \
#     'Insufficient prior time series data for extra_len_init. {} more is needed.'.format(df.index[0] - rows_train[0] + extra_len_init)
    
#     df_temp = df.loc[df_temp.index[0]-extra_len_init : df_temp.index[-1], ]    
#     # ....................................
    
#     df_temp = add_kama(df = df_temp, 
#                        src_colname = src_name, 
#                        delta_len = n_delta, 
#                        fast_len = n_fast, 
#                        slow_len = n_slow, 
#                        with_kama_mean = False, 
#                        kama_mean_len = kama_mean_len, 
#                        fillna = False)
    
#     ub_col = f'KAMA({src_name}, {n_delta}, {n_fast}, {n_slow})'
#     lb_col = f'KAMA-Mean({src_name}, {n_delta}, {n_fast}, {n_slow}, {kama_mean_len})'
#     kama_delta_col = f'delta({ub_col}, {delta_lim_len})'
    
#     kama_delta = df_temp[ub_col].rolling(delta_lim_len).apply(lambda x: delta(list(x), normalize = True))    # Caution: delta of KAMA, not of KAMA-Mean
#     df_temp[kama_delta_col] = kama_delta    # Caution: delta of KAMA, not of KAMA-Mean
    
#     # here 2022-11-11 1156
#     # drop the first extra_len_init rows of data
#     # these rows are just needed for prevention of NaN values in the indicator column(s) (ie KAMA and KAMA-Mean)
#     df_temp.drop(df_temp.index[:extra_len_init], inplace=True)
    
#     # buying / selling actions 
#     long_actions = df_temp[kama_delta_col] - delta_lim_up
#     long_actions = long_actions.apply(lambda x: 1 if x > 0 else None)
#     df_temp['long_actions'] = long_actions  

#     short_actions = df_temp[src_name] - df_temp[lb_col]
#     short_actions = short_actions.apply(lambda x: -1 if x < 0 else None)
#     df_temp['short_actions'] = short_actions
    
    # ==============================================================================================
    
#     # ==============================================================================================
#     # DC

#     global n_up_interval, n_dn_interval

#     # init
#     n_up, n_dn, bound_up_pct, bound_dn_pct = x    
#     n_up = int(round(n_up,0))    # ensure integer
#     n_dn = int(round(n_dn,0))    # ensure integer
    
#     n_up *= n_up_interval
#     n_dn *= n_dn_interval
    
#     # ----------------------------------
#     # add DC bounds
#     df_temp = add_dc(df = df_temp, 
#                      dc_source = 'Close', 
#                      dc_len = n_up, 
#                      bound_pct = 0, 
#                      bound = 'upper')
#     df_temp = add_dc(df = df_temp, 
#                      dc_source = 'Close', 
#                      dc_len = n_dn, 
#                      bound_pct = 0, 
#                      bound = 'lower')
#     ub_col = f'{n_up}-Period High'
#     lb_col = f'{n_dn}-Period Low'
    
#     # buying / selling actions 
#     long_actions = (df_temp['Close'] / df_temp[ub_col].shift(1) - 1) - bound_up_pct
#     long_actions = long_actions.apply(lambda x: 1 if x > 0 else None)
#     df_temp['long_actions'] = long_actions  

#     short_actions = (df_temp['Close'] / df_temp[lb_col].shift(1) - 1) - bound_dn_pct
#     short_actions = short_actions.apply(lambda x: -1 if x < 0 else None)
#     df_temp['short_actions'] = short_actions
    
#     # ==============================================================================================
    
    
    # integrate the buying and selling actions (ie a signal)
    # no delay is assumed
    actions = pd.Series(zip(long_actions, short_actions))
    del long_actions, short_actions
    
    actions = actions.apply(lambda x: -1 if x[1] == -1 else 1 if x[0] == 1 else 0)
    df_temp['actions'] = list(actions)    # must turn into a list so as to prevent future errors out of non-matching index
    
    del actions
    
    # debug here 2022-11-03 0004 WIP
    # testing

    # ----------------------------------------------------------------------------------------------
    # backtest
    # input
    initial_capital = 10**5
    long_rules = df_temp['actions'].apply(lambda x: 1 if x == 1 else 0).to_list()
    short_rules = df_temp['actions'].apply(lambda x: -1 if x == -1 else 0).to_list()
    buy_at = 'Close'
    commission_percent = 0.5
    buy_next_day = True
    
    # here 2022-12-04 1348 (risk_free_rate added)
    df_bt, txn_df, txn_summary = backtest(df = df_temp, 
                                          long_rules = long_rules,
                                          short_rules = short_rules,
                                          buy_at = buy_at,
                                          initial_capital = initial_capital, 
                                          commission_percent = commission_percent, 
                                          risk_free_rate = risk_free_rate, 
                                          buy_next_day = buy_next_day, 
                                          trading_days = trading_days)
    
    bt_return_pct = txn_summary['profit_net_pct']    # modified 2022-12-07 0954
    win_count = txn_summary['win_count']
    loss_count = txn_summary['loss_count']
    sharpe_annual = txn_summary['sharpe_delta_equity_annual']
    sortino_annual = txn_summary['sortino_delta_equity_annual']
    
    # ----------------------------------------------------------------------------------------------
    # objective function
    
    if obj_func_num == 1:    
        # version 1
        bt_ret = -bt_return_pct

    elif obj_func_num == 2:
        # version 2
        bt_ret = bt_return_pct_signflipped + - win_count**2
    
    elif obj_func_num == 3:
        # version 3
        bt_ret = - win_count**2 + loss_count**2
    
    elif obj_func_num == 4:
        # version 4
        bt_ret = - bt_return_pct - win_count**2 + loss_count**2
    
    elif obj_func_num == 5:
        # version 5
        # assign penalty if the Sortino ratio is NaN or None (because of insufficient trades)
        sortino_annual = sortino_annual if type(sortino_annual) == type(-1.0) else -10**6    
        bt_ret = - sortino_annual
    
    elif obj_func_num == 6:
        # version 6
        # assign penalty if the Sortino ratio is NaN or None (because of insufficient trades)
        sharpe_annual = sharpe_annual if type(sharpe_annual) == type(-1.0) else -10**6    
        bt_ret = - sharpe_annual
    
    elif obj_func_num == 7:
        # version 7
        # win_count + 1, loss_count + 1: the + 1 is for escaping undesirable effects due to zeros  
        bt_ret = - bt_return_pct * (win_count+1) / (loss_count+1)
    
    elif obj_func_num == 8:
    #     # version 8
    #     bt_ret =
        pass
    
    elif obj_func_num == 9:
        # version 9
        epsilon = 10**-6
        bt_ret = - bt_return_pct * (((win_count + epsilon) / (loss_count + epsilon)) ** (1 if bt_return_pct>0 else -1))
    
    elif obj_func_num == 10:
        # version 10
        bt_ret = - bt_return_pct * win_count
    
    elif obj_func_num == 11:
    #     # version 11
    #     bt_ret =
        pass
    
    elif obj_func_num == 12:
        # version 12
        bt_ret = - bt_return_pct * (win_count - loss_count)

    elif obj_func_num == 13:
        # version 13
        epsilon = 1
        bt_ret = - bt_return_pct * (((win_count + epsilon) / (loss_count + epsilon)) ** (1 if bt_return_pct>0 else -1))
    
    elif obj_func_num == 14:
        # version 14
        bt_ret = - bt_return_pct * (win_count - loss_count) * (-1 if bt_return_pct<0 and win_count<loss_count else 1)
    
    elif obj_func_num == 15:
        # version 15
        bt_ret = - (win_count - loss_count)**2 * (-1 if win_count<loss_count else 1)
    
    elif obj_func_num == 16:
        # version 16
        global bt_return_pct_lim    
        bt_ret = - ((win_count - loss_count)**2 * (-1 if win_count<loss_count else 1) + \
                    (bt_return_pct - bt_return_pct_lim)**2 * (-1 if bt_return_pct<bt_return_pct_lim else 1))
        print((bt_ret, (win_count, loss_count), (bt_return_pct, bt_return_pct_lim)))
        
    elif obj_func_num == 17:
        # version 17
        epsilon = 1
        bt_ret = - bt_return_pct * (((win_count + epsilon) / (loss_count + epsilon)) ** (2 if bt_return_pct>0 else -1))
    
    elif obj_func_num == 18:
        # version 18    
        bt_ret = - bt_return_pct * (win_count**2 - loss_count**2) * (-1 if bt_return_pct<0 and win_count<loss_count else 1)
    
    # here 2022-11-13
    # for determining the optimal value of the hyperparameter 'maxiter'
#     global opt_hist, temp_counter
    
#     opt_hist[-1].append(bt_ret)
#     print(f'{temp_counter}: {bt_ret}')

#     temp_counter += 1
    
    return bt_ret

In [ ]:
# ----------------------------------------------------------------------------------
# inputs
# inputs for results output and notifications in telegram
output_folder = r""
token = ''
chat_id =''

# inputs for data separation into training and testing set
rows_toview = 18077                        # define the start of the 1st period for training and testing
period_toview = 625                        # 1 yr (250 days); 2.5 years (625 days); 5 yrs (1250 days)
period_count = 40
train_pct = 0.8                            # normally taken as 0.7 ~ 0.8

# inputs for optimization
obj_func_num = 18
optim_model = 'DA'     # this is just for showing in the returning results
maxiter = 500
seed = 1
trading_days = 250

checker = 0    # a checker to ensure only 1 strategy is to be tested


# inputs for the bounds for optimization: 

# KAMA Only
# here 2022-12-04 1348 (risk_free_rate added)
# n_delta, n_fast, n_slow, delta_lim_up, delta_lim_dn, delta_lim_len

strat_name = 'KAMA Only'
risk_free_rate = 0
bounds = [(3, 20), 
          (2, 5), (10, 100), 
          (0, 0.1), 
          (-0.1, 0),
          (2, 5)]

extra_len_init = 1 + max(bounds[5][1], (max(bounds[2][1], bounds[1][1]) + bounds[0][1]))
    
checker += 1

# =======================================================================================================

# # KAMA & DC-low
# # n_delta, n_fast, n_slow, delta_lim_up, delta_lim_len, n_dn

# strat_name = 'KAMA with DC-low (No dn_pct)'
# bounds = [(3, 20), 
#           (2, 5), (10, 100), 
#           (0, 0.1), 
#           (2, 5), 
#           (3, 20)]

# # here 2022-11-11 1156
# # consider extra prev data to generate values of KAMA-Mean in the beginning days so as to prevent NaN in these days
# # max(n_dn, (max(n_slow, n_fast) + n_delta))
# extra_len_init = 1 + max(bounds[5][1], (max(bounds[2][1], bounds[1][1]) + bounds[0][1]))
    
# checker += 1

# =======================================================================================================
# KAMA & KAMA-Mean
# n_delta, n_fast, n_slow, delta_lim_up, delta_lim_dn, delta_lim_len, kama_mean_len
# strat_name = 'KAMA with KAMA-Mean'
# bounds = [(3, 20), 
#           (2, 5), (10, 100), 
#           (0, 0.1), (0, 0.1), 
#           (2, 5), (5, 100)]

# # here 2022-11-11 1156
# # consider extra prev data to generate values of KAMA-Mean in the beginning days so as to prevent NaN in these days
# # kama_mean_len + delta_lim_len + max(n_slow, n_fast, n_delta)
# extra_len_init = 1 + bounds[6][1] + bounds[5][1] + max(bounds[0][1], bounds[1][1], bounds[2][1])
    
# checker += 1
# # =======================================================================================================
# # DC
# # inputs for the bounds for optimization: 
# # n_up (to be multiplied by the interval), n_dn (to be multiplied by 10), bound_up_pct, bound_dn_pct
# strat_name = 'DC'
# bounds = [(1, 20), (1, 20), 
#           (0, 0.1), (0, 0.1)]

# n_up_interval = 5
# n_dn_interval = 5

# checker += 1

# =======================================================================================================

# ----------------------------------------------------------------------------------
assert checker == 1., f"Only 1 stategy can be tested each time, but there are {checker} detected! Check the codes corresponding to the inputs part."
del checker

print(f'''
token: {token}
chat_id: {chat_id}
''')

# param_info = f'''
# XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

# strat_name: {strat_name}
# dt.datetime.now(): {dt.datetime.now()}

# Param.

# output_folder: {output_folder}
# rows_toview: {rows_toview}
# period_toview: {period_toview}
# train_pct: {train_pct}
# maxiter: {maxiter}
# seed: {seed}
# bounds: {bounds}
# '''

param_info = f'''
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

strat_name: {strat_name}
dt.datetime.now(): {dt.datetime.now()}

Param.

output_folder: {output_folder}
rows_toview: {rows_toview}
period_toview: {period_toview}
train_pct: {train_pct}
optim_model: {optim_model}
maxiter: {maxiter}
seed: {seed}
bounds: {bounds}
extra_len_init: {extra_len_init}
'''
print(param_info)

# print('\nXXXXXXXXfXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX\n')
# print(f'strat_name: {strat_name}\n')
# print('Param.\n')
# print(f'output_folder: {output_folder}')
# print(f'rows_toview: {rows_toview}')
# print(f'period_toview: {period_toview}')
# print(f'train_pct: {train_pct}')
# print(f'maxiter: {maxiter}')
# print(f'bounds: {bounds}')



# ----------------------------------------------------------------------------------
# init.

period_test = int(round((1-train_pct)*period_toview,0))

# get intervals for traing and testing
print('\nXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX\n')
print('\nIntervals of Periods: \n')

periods = [((rows_toview+period_test*i, df.loc[rows_toview+period_test*i, 'Date']), 
            (rows_toview+period_test*i+period_toview-1, df.loc[rows_toview+period_test*i+period_toview-1, 'Date'])) for i in range(period_count)]
# # Superseded
# # Originally, in the trial, overall periods for training and testing are defined as below:
# periods = [((rows_toview+period_toview*i, df.loc[rows_toview+period_toview*i, 'Date']), 
#             (rows_toview+period_toview*(i+1)-1, df.loc[rows_toview+period_toview*(i+1)-1, 'Date'])) for i in range(9)]


for p in range(len(periods)):
    print((p, periods[p]))

In [ ]:
# send the param_info in telegram
send_to_telegram(message = param_info, 
                 token=token, 
                 chat_id=chat_id)

##### Get % return for each testing and training period if
##### << the given strategy is used >>

In [ ]:
# here 2022-10-31

# init
results = dict()

# here 2022-11-13
# for determining the optimal value of the hyperparameter 'maxiter'
opt_hist=[]
temp_counter = 0

# loop through different intervals for training and testing

# for p in tqdm(range(len(periods)-1, 0-1, -1), token=token, chat_id=chat_id):    # doing in reversed order
for p in tqdm(range(0, int(len(periods)/2)), token=token, chat_id=chat_id):             # doing in order
    
    # send a telegram message
    send_to_telegram(message = f'Overall Period: {periods[p]} now ({dt.datetime.now()})', token=token, chat_id=chat_id)
    
    # here 2022-11-13
    # for determining the optimal value of the hyperparameter 'maxiter'
    opt_hist.append([])

    # get rows for training and testing
    rows_train = periods[p][0][0], periods[p][0][0] + int(period_toview*train_pct)
    rows_test = rows_train[-1] + 1, periods[p][1][0]

    print('\nXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
    print(p)
    print(f'Overall Period: {periods[p]}')
    print('\n~~~ Training : from {} to {}~~~\n'.format(df.loc[rows_train[0], 'Date'], df.loc[rows_train[1], 'Date'])) 
    
    df_summary = df.loc[rows_train[0]: rows_test[1], ['Date', 'Close', 'Close_ln']].copy()
    df_train = df_summary.loc[rows_train[0]:rows_train[1],]
    df_test = df_summary.loc[rows_test[0]:rows_test[1],]

    # ----------------------------------------------------------------------------------
    # Training set
    # optimizate param using the df_train
    start_time = dt.datetime.now()
    
    df_temp = df_train.copy()
    bt_return_pct_lim = 100*(df_temp.loc[df_temp.index[-1],'Close'] / df_temp.loc[df_temp.index[0],'Close'] - 1)
    
    if optim_model == 'DA':
        optim_results = optimize.dual_annealing(func = objective_func, bounds = bounds, maxiter = maxiter, seed = seed)
    elif optim_model == 'diff evol':
        optim_results = optimize.differential_evolution(func = objective_func, bounds = bounds, maxiter = maxiter, seed = seed)
        
    end_time = dt.datetime.now()
    runtime = end_time - start_time
    optim_results['x'] = list(optim_results['x'])    # for saving in json format
    print('Obj Func Ret: {}'.format(optim_results['fun']))

    # save results to a variable
    date_from = df_train.loc[rows_train[0], 'Date']
    date_to = df_train.loc[rows_train[1], 'Date']
    data_type = 'Training'
    result_entryname_train = '{}_{}_{}_{}'.format(optim_model,data_type, date_from, date_to)
    
#     results[result_entryname_train] = {
#         'Type': data_type, 
#         'From': str(df_train.loc[rows_train[0], 'Date']), 
#         'To': str(df_train.loc[rows_train[1], 'Date']),
#         'Maxiter': maxiter,
#         'Bounds': bounds,
#         'Result': optim_results,
#         'Runtime': str(runtime), 
#         'Runtime Per Iter': str(runtime/maxiter)}

    results[result_entryname_train] = {
        'Strategy': strat_name,
        'Type': data_type, 
        'From': str(df_train.loc[rows_train[0], 'Date']), 
        'To': str(df_train.loc[rows_train[1], 'Date']),
        'Maxiter': maxiter,
        'Bounds': bounds,
        'Result': optim_results,
        'Runtime': str(runtime), 
        'Runtime Per Iter': str(runtime/maxiter),
        'TXN Summary': txn_summary, 
        'TXN log': txn_df.to_dict()}

    print(f'\n{data_type} Completed !\n')
    print('Runtime: {}\n'.format(results[result_entryname_train]['Runtime']))
    
    # save as json
    
    save_as_json(inp_dict = results[result_entryname_train], 
             output_folder = output_folder, 
             indent = 4, 
             output_filename_prefix = '{} Results_{}'.format(strat_name, result_entryname_train))
    
    # ----------------------------------------------------------------------------------
    # Testing set
    # get param for the obj func for the testing set
    result_touse = result_entryname_train
    param_test = results[result_touse]['Result']['x']

    # get result using the param retrieved
    start_time = dt.datetime.now()
    df_temp = df_test.copy()
    obj_func_ret = objective_func(param_test)
    end_time = dt.datetime.now()
    runtime = end_time - start_time
    
    print('Obj Func Ret: {}'.format(obj_func_ret))
    
    # save results to a variable
    date_from = df_test.loc[rows_test[0], 'Date']
    date_to = df_test.loc[rows_test[1], 'Date']
    data_type = 'Testing'
    result_entryname_test = '{}_{}_{}_{}'.format(optim_model,data_type, date_from, date_to)

    results[result_entryname_test] = {
        'Strategy': strat_name,
        'Type': data_type, 
        'From': str(df_train.loc[rows_train[0], 'Date']), 
        'To': str(df_train.loc[rows_train[1], 'Date']),
        'Maxiter': maxiter,
        'x': param_test,
        'Obj Func Ret': obj_func_ret,
        'Runtime': str(runtime), 
        'Runtime Per Iter': str(runtime/maxiter),
        'TXN Summary': txn_summary, 
        'TXN log': txn_df.to_dict()}

    print(f'\n{data_type} Completed !\n')
    print('Runtime: {}\n'.format(results[result_entryname_test]['Runtime']))
    
    # save as json
    save_as_json(inp_dict = results[result_entryname_test], 
             output_folder = output_folder, 
             indent = 4, 
             output_filename_prefix = '{} Results_{}'.format(strat_name, result_entryname_test))
#     break

In [ ]:
save_as_json(inp_dict = results, 
             output_folder = output_folder, 
             indent = 4, 
             output_filename_prefix = f'{strat_name} Results')

In [ ]:
send_to_telegram(message = f'''
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Execution completed.
All results have been saved to {output_folder}.
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
''', 
                 token=token, 
                 chat_id=chat_id)

In [ ]:
opt_hist_df = pd.DataFrame(opt_hist).transpose()

# ------------------------------------------------------------
# input
try:
    opt_hist_df.columns = periods[0 : int(len(periods)/2)]
except:
    opt_hist_df.columns = [str(i) for i in periods[0 : int(len(periods)/2)]]

opt_hist_df

In [ ]:
save_as_csv(input_df = opt_hist_df, 
            output_folder = output_folder, 
            output_filename_prefix = 'opt_hist_df')